## Man In the Middle Attack
Consider the sequence of events below:
1. Bob generates keypair and sends its public key to Alice
2. Eve eavesdrop the conversations and captures the public key and replaces Bob's public key with her own
3. Bob sends a message and its signature to Alice
4. Eve changes the message and signs with her own private key
5. Alice receives the message and its verifies its signature thinking the message came from Bob

One possible way to resolve this is to meet in person to exchange key, but this is not always possible. Another way is to use digital certificates. The certificate will contain Bob's public key and his identity. If the certificate was signed by a CA, Alice can verify that the public key is actually Bob's.

## Digital Certificate
Is a data structure used to identity presentation and verification. It is a digital identity card. A digital certificate contains:
- Owner information
- Public key: owner's public key
- Issuer: the entity who signed the certificate
- Validity dates: dates through which the certificate is considered valid
- Signature: signed by the issuer

Thus a digital certificate is owner information, their public key and some additional metadata (like validity) signed by an issuing authority. It binds a public key to an identity. To validate the digital certificate use the issuer's public key. The issuer is generally a CA which is trusted by everyone involved and various entities involved have a pre-trusted listed of issuer's public keys.

There are different digital certificate formats like:
- X.509
- PGP certificate
- SSH certificate
X.509 is the most commonly used and is discussed later.

## X.509 Certificate
Has the following content:
```
Certificate:
    Version: 3
    Serial Number: 0B:D8:F6:37:B6:5E:A9:52:D9:BB:3D:AB:FD:73:45:06
    Certificate Signature Algorithm: PKCS #1 SHA-256 With RSA Encryption
    Issuer: CN = DigiCert SHA2 High Assurance Server CA, OU = www.digicert.com, O = DigiCert Inc, C = US
    Validity:
        Not Before: 03/12/2025, 05:30:00 GMT+5:30
        Not After: 22/01/2026, 05:29:59 GMT+5:30
    Subject: CN = *.yahoo.com, O = Yahoo Holdings Inc., L = New York, ST = New York, C = US
    Subject Public Key Info:
        Subject Public Key Algorithm: Elliptic Curve Public Key
        Subject Public Key: 00 04 4C 49 D5 2E 2C 64 C1 A2 FC 13 BF D4 98 EC ...
    Extensions:
        Certificate Key Usage: 
            - Critical
            - Signing
            - Key Agreement
        Certificate Subject Alternative Name:
            - DNS Name: *.yahoo.com
            - DNS Name: *.www.yahoo.com
            - DNS Name: ymail.com
            - DNS Name: s.yimg.com
            ...
        ...
    Certificate Signature Algorithm: PKCS #1 SHA-256 With RSA Encryption
    Certificate Signature Value: 6F D3 49 3A AB 3B C4 46 CA 73 43 DF B0 9C B6 28 ...
    SHA-256 Fingerprints:
        Certificate: 18a39c6b4d4c7bb45434ac7eb000db9d8903b3a97f5f2a1ba9624967873ef08c
        Public Key: 4b590e6e4f04bf7eb47833ecd116bc42eb22e85a1f5d6643b8a6e521edef5aff
```

**Serial Number:** unique number of the certificate among certificates signed by the same issuer. Can be used to selectively revoke certificates.  
**Signature Algorithm:** algorithm used by issuer to sign the certificate.  
**Issuer:** details in DN (Distinguished Name) format. Its various components are explained below:  
- **CN (Common Name):** can be a person's name, a website's domain name, etc.
- **OU (Organizational Unit):** subdivision within an organization like "IT", "Security", etc.
- **O (Organization):** legal organization name, like "University of XYZ", *Google Inc.", etc.
- **L (Locality):** city or locality like "London" or "Paris".
- **ST (State):** state, province or region like "Texas" or "Punjab".
- **C (Country):** two letter country code like "US" or "IN".
  
**Validity:** time duration when the certificate is considered valid.  
**Subject:** the entity that the certificate identifies in DN format. It can be a person, a website, an organization, or something else.  
**Subject Public Key Info:** information about subject’s public key.  
**Extensions:** additional information included in the certificate. An important one is SAN (Subject Alternative Name) which is used in TLS connections. It lists additional identities a certificate is valid for (example additional domains this certificate is valid for).  
**Signature Algorithm:** algorithm used by issuer to sign this certificate.  
**Signature Value:** signature produced by the signature algorithm.

### Certificate Authority
CA is an organization that issues certificates after verifying the identity mentioned in the subject. The idea is that we trust CAs to issue certificates and consider certificates issued by those CAs valid. Operating systems and browsers come bundled with a list of trusted root CA certificates forming a trust store. These certificates are self-signed, and their public keys are trusted a priori and used to validate the signatures on intermediate and end-entity certificates. An example of root certificate:
```
Certificate:
    Version: 3
    Serial Number: 02:AC:5C:26:6A:0B:40:9B:8F:0B:79:F2:AE:46:25:77
    Certificate Signature Algorithm: PKCS #1 SHA-256 With RSA Encryption
    Issuer: CN = DigiCert High Assurance Server EV Root CA, OU = www.digicert.com, O = DigiCert Inc, C = US
    Validity:
        Not Before: 10/11/2006, 05:30:00 GMT+5:30
        Not After: 10/11/2031, 05:30:00 GMT+5:30
    Subject: CN = DigiCert High Assurance Server EV Root CA, OU = www.digicert.com, O = DigiCert Inc, C = US
    Subject Public Key Info:
        Subject Public Key Algorithm: PKCS #1 RSA Encryption
        Subject Public Key: Modulus (2048 bits), C6 CC E5 73 E6 FB D4 BB E5 2D 2D 32 A6 DF E5 81 ...
    Extensions:
        Certificate Key Usage: 
            - Critical
            - Signing
            - Certificate Signer
            - CRL Signer
        ...
    Certificate Signature Algorithm: PKCS #1 SHA-256 With RSA Encryption
    Certificate Signature Value: 1C 1A 06 97 DC D7 9C 9F 3C 88 66 06 08 57 21 DB ...
    SHA-256 Fingerprints:
        Certificate: 7431e5f4c3c1ce4690774f0b61e05440883ba9a01ed00ba6abd7806ed3b118cf
        Public Key: 5a889647220e54d6bd8a16817224520bb5c78e58984bd570506388b9de0f075f
```
The subject and issuer is the same indicating this is self-signed.

### Certificate Chain
How can we verify the Yahoo certificate? It needs the issuer *DigiCert SHA2 High Assurance Server CA*'s public key which is bundled in the issuer certificate called the intermediate certificate. But that is not a root certificate, so we need to validate it. We look at this intermediate certificate's issuer which is *DigiCert High Assurance Server EV Root CA*. Now this is a root certificate, so we trust it and use its public key to validate the intermediate certificate.

<img src="images/cert_chain.png" width=900 height=auto />

Why do we need intermediate certificates? Intermediate certificates are issued by intermediate CA. The root CAs are kept in an offline system and are rarely used to sign, since if it gets compromised the entire infrastructure will be compromised. Thus we use these intermediate CA who do most of the signing duty. This way we limit the failure radius and distribute signing load among multiple intermediate CAs rather than overburdening few root CAs.

## Certificate Signing Request
To obtain a certificate signed by a CA, we first need to generate CSR. It contains information about the subject and the public key. The CSR is signed by the subject's private key. This establishes that the public key present in the certificate is owned by the subject. The CSR is sent to CA which does some validation to ensure the applicant is who he says he is. The CA also adds other information to the certificate, such as the issuer, validity fields, and X509v3 extensions. Finally, the CA signs the certificate.

**Domain Validated:** while issuing certificate for a website, CAs do not do a thorough check of the website and its owners, scan the website for malware, visit the website's owners in person, check a lot of their documents, and so on. The owner just has to prove that he owns the domain. These certificates are domain-validated certificates.

**Extended Validation:** in this case many more checks are done like verifying governement issued IDs, etc. EV certificates are only issued to organizations, not to individuals.

**Individual Validation and Organization Validation:** between DV and EV.

## OpenSSL
To verify a certificate, supply all certificates involved in the chain:
```bash
$ openssl verify -verbose -show_chain -trusted root-cert.pem -untrusted inter-cert.pem leaf-cert.pem leaf-cert.pem: OK
Chain:
depth=0: C=US, ST=New York, L=New York, O=Yahoo Holdings Inc., CN=*.yahoo.com (untrusted)
depth=1: C=US, O=DigiCert Inc, OU=www.digicert.com, CN=DigiCert SHA2 High Assurance Server CA (untrusted)
depth=2: C=US, O=DigiCert Inc, OU=www.digicert.com, CN=DigiCert High Assurance EV Root CA
```